In [1]:
import urllib.request
import numpy as np
import json
import string
from sklearn import metrics as sk_m
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import sklearn as sk
import matplotlib.pyplot as plt
import random as random
from scipy.stats import norm
import statistics as stat
from weat import weat

### Load original embedding

In [2]:
# URL to retrive pre-trained 300 dimensional gloVe embedding
embedding_300_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/vectors.txt"

def read_embedding(url, skip_first = False):
    """Function to read out an embedding
    Input: url: url to embedding
    
    Returns: vocab: list of words in the embedding
             w2id: dictionary mapping words to ids
             embedding: array storing the word vectors,
                           row corresponds to word id"""
    # Open url
    data = urllib.request.urlopen(url)
    vocab = []
    embedding = []
    
    # Each line contains one word and its embedding
    for i, line in enumerate(data):
        if skip_first:
            if i == 0:
                continue
        #if len(line) == 301:
        line = line.decode()
        # Split by spaces
        split = line.split()
        # First element(== the word) is added to vocabulary
        vocab.append(split[0])
        # All other elements(embedding vectors) are added to vectors
        embedding.append([float(elem) for elem in split[1:]])
    
    # Create a dictionary with word-id pairs based on the order
    w2id = {w: i for i, w in enumerate(vocab)}
    # Vectors are converted into an array
    embedding = np.array(embedding).astype(float)
    
    return vocab, w2id, embedding
    
vocab_original, w2id_original, embedding_original = read_embedding(embedding_300_url)

In [3]:
def hasDigit(word):
    """Checks if a string contains any digits"""
    return any(char.isdigit() for char in word)

def hasSpecialChar(word):
    """Checks if a string contains special characters(except "_")"""
    special_characters = "!@#$%^&*()-+?=,<>/."
    return any(char in special_characters for char in word)

In [30]:
def restrict_vocab(vocab, w2id, embedding):
    """Limits the vocab by removing words containing digits or special characters
    Input: vocab: list of words in the embedding
           w2id: dictionary mapping words to ids
           embedding: array storing the word vectors
           
    Returns: limit_vocab: list of words in vocab that do not include digits or special characters
             limit_w2id: dictionary mapping words in limit_vocab to new ids
             limit_embedding: array storing the word vectors of the words in limit_vocab only"""
    limit_vocab = []
    limit_embedding = []
    
    for i, word in enumerate(vocab[:75000]): # hoping that this gives us the most common words
        # If word includes either a digit or a special character move on to next word
        if (hasDigit(word) or hasSpecialChar(word)):
            continue
        # Else add word to limit_vocab and its embedding to limit_embedding    
        limit_vocab.append(word)
        limit_embedding.append(embedding[w2id[word]])
        
    # Convert embedding into an array    
    limit_embedding = np.array(limit_embedding).astype(float)
    # Create new dictionary containing only the words in limit_vocab and their new ids
    limit_w2id = {word: i for i, word in enumerate(limit_vocab)}
    
    return limit_vocab, limit_w2id, limit_embedding

In [31]:
vocab, w2id, embedding = restrict_vocab(vocab_original, w2id_original, embedding_original)
print("Original vocab size: ", len(vocab_original))
print("Restricted vocab size: ", len(vocab))

Original vocab size:  322636
Restricted vocab size:  72306


In [32]:
def exclude_vocab(vocab, exclude):
    """Function to exclude specific words from vocabulary
    Input: vocab: list of words in the embedding
           exclude: list of words to exclude from the vocabulary
           
    Returns: limited_vocab: vocab without the words in exclude"""
    # Create copy of vocab
    limited_vocab = vocab.copy()
    # For all words that are in exclude and vocab
    for word in exclude:
        if word in limited_vocab:
            # Remove word from vocab
            limited_vocab.remove(word)
            
    return limited_vocab

In [33]:
# URL to female specific words as listed by the authors
female_words_url = "https://raw.githubusercontent.com/uvavision/Double-Hard-Debias/master/data/female_word_file.txt"
female_words_data = urllib.request.urlopen(female_words_url)

# List of female words
female_words = []
for line in female_words_data:
    line = line.decode()
    line = line.split()
    female_words.append(line[0])

In [34]:
# URL to male specific words as listed by the authors
male_words_url = "https://raw.githubusercontent.com/uvavision/Double-Hard-Debias/master/data/male_word_file.txt"
male_words_data = urllib.request.urlopen(male_words_url)

# List of male words
male_words = []
for line in male_words_data:
    line = line.decode()
    line = line.split()
    male_words.append(line[0])

In [35]:
# Create List with female - male pairs from female-male specific words
female_male_pairs = []
for i, female in enumerate(female_words):
    female_male_pairs.append([female, male_words[i]])

In [36]:
# URLs to the files storing gender specific words as listed by the authors
gender_specific_url = "https://raw.githubusercontent.com/uvavision/Double-Hard-Debias/master/data/gender_specific_full.json"

# Empty list to accumulate gender specific words plus additional list after lowercasing
gender_specific_original = []
gender_specific = []


# Read out URL and add further gender specific words
with urllib.request.urlopen(gender_specific_url) as f:
    gender_specific_original.extend(json.load(f))

# Add lower case words to second list
for word in gender_specific_original:
    gender_specific.append(word.lower())

In [37]:
# URL to the file storing definitional pairs as listed by the authors
definitial_pairs_url = "https://raw.githubusercontent.com/uvavision/Double-Hard-Debias/master/data/definitional_pairs.json"

# Empty list to store definitional pairs plus additional list after lowercasing
definitional_pairs_original = []
definitional_pairs = []


# Read out url and add pairs in list
with urllib.request.urlopen(definitial_pairs_url) as f:
    definitional_pairs_original.extend(json.load(f))
    
# Add lower case pairs to second list
for [w1, w2] in definitional_pairs_original:
    definitional_pairs.append([w1.lower(), w2.lower()])


# Create list of single words instead of pairs  
definitional_words = []
for pair in definitional_pairs:
    for word in pair:
        definitional_words.append(word)
        

In [38]:
# URL to the file storing the equalize pairs as listed by the authors
equalize_pairs_url = "https://raw.githubusercontent.com/uvavision/Double-Hard-Debias/master/data/equalize_pairs.json"

# Empty list to store equalize pairs plus additional list after lowercasing
equalize_pairs_original = []
equalize_pairs = []

# Read out URL and add pairs to list
with urllib.request.urlopen(equalize_pairs_url) as f:
    equalize_pairs_original.extend(json.load(f))
    
# Add lower case pairs to second list
for [w1, w2] in equalize_pairs_original:
    equalize_pairs.append([w1.lower(), w2.lower()])
    
# Create list of single words instead of pairs
equalize_words = []
for pair in equalize_pairs:
    for word in pair:
        equalize_words.append(word)

In [39]:
# List of all gender specific words included in 
# female words, male words, gender specific words, equalize words and definitional words
exclude_words = list(set(female_words + male_words + gender_specific + definitional_words + equalize_words))

In [40]:
def embed(word, w2id=w2id, embedding=embedding):
    return embedding[w2id[word]]

In [41]:
# Remove gender specific words from the embedding to obtain vocabulary of neutral words
vocab_neutral = exclude_vocab(vocab, exclude_words)
# save both neutral embeddings and the indices of the neutral words in original vocab
embedding_neutral = np.asarray([embed(word) for word in vocab_neutral])
id_neutral = [w2id[neutral] for neutral in vocab_neutral]

print("Vocab size: ", len(vocab))
print("Neutral vocab size: ", len(vocab_neutral), embedding_neutral.shape, len(id_neutral))

Vocab size:  72306
Neutral vocab size:  71858 (71858, 300) 71858


### Load further embeddings

In [19]:
embedding_gn_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/vectors300.txt"
vocab_gn_original, w2id_gn_original, embedding_gn_original = read_embedding(embedding_gn_url)
vocab_gn, w2id_gn, embedding_gn = restrict_vocab(vocab_gn_original, w2id_gn_original, embedding_gn_original)

In [20]:
def debias_gn(wv):
    for v in wv:
        assert(len(v) == 300)
    
    wv = wv[:,:-1]

    for v in wv:
        assert(len(v) == 299)
    return wv

vocab_gn_a = vocab_gn
w2id_gn_a = w2id_gn
embedding_gn_a = debias_gn(embedding_gn)

In [21]:
embedding_hd_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/vectors_hd.txt"
vocab_hd_original, w2id_hd_original, embedding_hd_original = read_embedding(embedding_hd_url)
vocab_hd, w2id_hd, embedding_hd = restrict_vocab(vocab_hd_original, w2id_hd_original, embedding_hd_original)

In [ ]:
embedding_hd_a_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/vectors_hd_a.txt"
vocab_hd_a_original, w2id_hd_a_original, embedding_hd_a_original = read_embedding(embedding_hd_a_url)
vocab_hd_a, w2id_hd_a, embedding_hd_a = restrict_vocab(vocab_hd_a_original, w2id_hd_a_original, embedding_hd_a_original)

In [ ]:
embedding_gp_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/gp_glove.txt"
vocab_gp_original, w2id_gp_original, embedding_gp_original = read_embedding(embedding_gp_url, skip_first = True)
vocab_gp, w2id_gp, embedding_gp = restrict_vocab(vocab_gp_original, w2id_gp_original, embedding_gp_original)

In [38]:
embedding_gp_gn_url = "http://www.cs.virginia.edu/~tw8cb/word_embeddings/gp_gn_glove.txt"
vocab_gp_gn_original, w2id_gp_gn_original, embedding_gp_gn_original = read_embedding(embedding_gp_gn_url, skip_first = True)
vocab_gp_gn, w2id_gp_gn, embedding_gp_gn = restrict_vocab(vocab_gp_gn_original, w2id_gp_gn_original, embedding_gp_gn_original)

Original vocab size:  322636
Restricted vocab size:  314952


### Gender Subspace

In [42]:
def idtfy_gender_subspace(word_sets, w2id, defining_sets, k=1):
    """
    identifies the bias (gender) subspace following Bolukbasi et al. 2016
    
    takes
    word_sets: vocabulary
    w2id: a dictionary to translate words contained in the vocabulary into their corresponding IDs
    defining_sets: N defining sets (pairs if I=2) consisting of I words that differ mainly on the bias (gender) direction
    embedding: the embedding of the vocabulary
    k: an integer parameters that defines how many rows of SVD(C) constitute the bias (gender) subspace B, bias (gender) direction if k=1
    
    returns
    bias_subspace: linear bias (gender) subspace (direction) that is assumed to capture most of the gender bias (denoted as B in Bolukbasi et al. 2016)
    """
    
    C = []
    for female_word, male_word in defining_sets:
        mean = (embed(female_word) + embed(male_word)) /2
        C.append(embed(female_word) - mean)
        C.append(embed(male_word) - mean)
    C = np.array(C)
    
    # applying PCA is the same as SVD when interpreting C as covariance matrix (Vargas & Cotterell 2020)
    pca = PCA(n_components = 10)
    pca.fit(C)

    # take the first k pcas (first for gender direction)
    B = []
    for i in range(k):
        B.append(pca.components_[i])
    B = np.array(B).flatten()

    
    #print("new_B")
    #array = np.ndarray((10,2,300))
    #i=0
    #array_two = np.zeros((10,300,300))
    #for j, d_pair in enumerate(definitional_pairs):
    #    for i, word in enumerate(d_pair):
    #        # fill array with embeddings
    #        array[j][i] = embedding[w2id[word]]
            #i = i+1
        # print(array[j][0].shape)
        # calculate covariance between embeddings of same definitional pair?
    #    array_two[j]=np.cov(np.transpose(array[j]))
        
    
    return B

In [66]:
gender_subspace = idtfy_gender_subspace(vocab, w2id, definitional_pairs)

### most biased 500 words

In [44]:
# most biased male and female words
def most_biased(embedding, B, k=500):
    # small x, else memory issues
    #x = 50000
    all_biased = np.ndarray((len(embedding),1))
    for i, word in enumerate(embedding):
        #if i < x:
        all_biased[i] = (sk_m.pairwise.cosine_similarity(word.reshape(1, 300), B.reshape(1, 300)))[0]
            # print(sk_m.pairwise.cosine_similarity(word.reshape(1,300), B)[0])
    #print(all_biased)
    most_biased_f = []
    most_biased_m = []
    for word in range(k):
        # female words
        fb_index = np.argmin(all_biased)
        most_biased_f.append(fb_index)
        all_biased[fb_index] = 0
        # male words
        mb_index = np.argmax(all_biased)
        most_biased_m.append(mb_index)
        all_biased[mb_index] = 0
    #print(most_biased_f, most_biased_m)
    return most_biased_f, most_biased_m

In [72]:
index_f, index_m = most_biased(embedding, gender_subspace)

In [74]:
female_most_biased = [vocab[i] for i in index_f]
print("female", female_most_biased) #switched again???
male_most_biased = [vocab[i] for i in index_m]
print("male", male_most_biased)

female ['actress', 'pregnant', 'louise', 'therese', 'voluptuous', 'abbess', 'ivanovna', 'sister', 'chairwoman', 'alumna', 'princess', 'ballerina', 'foundress', 'maid', 'headmistress', 'pregnancy', 'englishwoman', 'josephine', 'olga', 'spinster', 'businesswoman', 'socialite', 'woman', 'ivanova', 'heroine', 'councilwoman', 'petrovna', 'congresswoman', 'hippolyta', 'matron', 'emmeline', 'prioress', 'seductive', 'uterus', 'feminist', 'actresses', 'feisty', 'princesses', 'mary', 'herself', 'headscarf', 'vissi', 'suffragette', 'suffragist', 'louisa', 'ellen', 'countess', 'waitress', 'emily', 'goddess', 'girl', 'aunt', 'menstruation', 'sisters', 'alena', 'cecile', 'governess', 'agnes', 'laura', 'duchess', 'filipina', 'archduchess', 'kuznetsova', 'daughters', 'comtesse', 'svetlana', 'menstrual', 'assunta', 'noblewoman', 'vivacious', 'katherine', 'mother', 'grandmother', 'elsa', 'nun', 'mauresmo', 'handbag', 'rebecca', 'prostitute', 'marchioness', 'sophie', 'valentina', 'irene', 'girlfriend', '

### Double Hard Debias

The vector projection of a vector a on (or onto) a nonzero vector b, (also known as the vector component or vector resolution of a in the direction of b), is the orthogonal projection of a onto a straight line parallel to b. It is a vector parallel to b, defined as:

${\displaystyle \mathbf a_{1}} =  a_{1} {\mathbf {\hat {b}} \,} $

where $a_{1}$ is a scalar, called the scalar projection of a onto b, and ${\mathbf {\hat {b}} \,}$ is the unit vector in the direction of b.

In turn, the scalar projection is defined as:[2]

$    a_{1} =  a ⋅ {\mathbf {\hat {b}} \,}= a ⋅ \frac{b}{‖ b ‖} $

where the operator ⋅ denotes a dot product, ‖b‖ is the length of b.

The **vector component** or vector resolute of a perpendicular to b, sometimes also called the **vector rejection of a from b** is the orthogonal projection of a onto the plane (or, in general, hyperplane) orthogonal to b. Both the projection $a_{1}$ and rejection $a_{2}$ of a vector a are vectors, and their sum is equal to a,[1] which implies that the rejection is given by: 

$a_{2} = a − a_{1}$ 

In [75]:
def double_hard_debias(embedding, embedding_neutral, id_neutral, index_m, index_f):
    """
    Double Hard Debias as proposed by Wang et. al.:
    
    takes
    embedding: all embeddings
    embedding_neutral: subset of embeddings that the bias should be removed from
    index_m: indices of most biased male words 
    index_f: indices of most biased female words 
    
    returns
    double_debias: full set of double-debiased embeddings
    """
    
    # create word lists of most biased male and female words
    males = np.asarray([embedding[i] for i in index_m])
    females = np.asarray([embedding[i] for i in index_f])    
    
    # decentralize all of the embeddings and store seperately
    words_decen = np.zeros((len(embedding),300), dtype='float32') # chose a smaller data type due to memory error
    words_neutral_decen = np.zeros((len(embedding_neutral),300), dtype='float32')
    # first calculate mean over full vocab
    mu = ((len(embedding)**(-1)) * np.sum(embedding, axis=0)).reshape(1,300)
    # then subtract mean from each word embedding
    for index, emb in enumerate(embedding):
        words_decen[index] = emb - mu
        
    for index, emb in enumerate(embedding_neutral):
        words_neutral_decen[index] = emb - mu
    
    #print("decentralized:",words_decen.shape)
    #print("origin:",words)
        
    # discover the frequency direction    
    # for all decentralized embeddings: compute PCA
    #print("Principal Components:",princ_comp)
    pca_freq = PCA().fit(words_decen)
    # print(pca.components_)
    
    evaluations = []

    # in implementation of paper only consider 20 first PCs
    for i, pc in enumerate(pca_freq.components_):
        if i < 20:
            male_proj = np.zeros((len(males),300))
            male_debias = np.zeros((len(males),300))
            female_proj = np.zeros((len(females),300))
            female_debias = np.zeros((len(females),300))
    
        
            #pc= pc.reshape(1,300)
        
            # remove PC direction and gender direction from all embeddings
            for index, male in enumerate(males):
                
                # remove direction of current PC
                male_proj[index] = w_orth(male, pc)
                # remove gender direction: hard debias
                male_debias[index] = w_orth(male_proj[index], gender_subspace)
            
            # repeat for female-biased words
            for index, female in enumerate(females):
            
                female_proj[index] = w_orth(female, pc)
                female_debias[index] = w_orth(female_proj[index], gender_subspace)
    
            # apply Neighbourhood Metric
            # compute gender alignment accuracy for each PC
            evaluations.append(align_acc(male_debias, female_debias))
            
    print("eval:",evaluations)
    
    # evaluate which PC-rejection leads to most random cluster = evaluation smallest (closest to 0.5) 
    # in original paper corresponded to second PC    
    print(np.argmin(evaluations))
    best_pc = pca_freq.components_[np.argmin(evaluations)]

    first_debias = np.zeros((embedding.shape))
    first_neutral_debias = []
    # remove best PC-direction from all neutral, decentralized words
    for index,word in enumerate(words_decen):
        
        if index in id_neutral:
            first_debias[index] = w_orth(word, best_pc)
            first_neutral_debias.append(w_orth(word, best_pc))
        else:
            first_debias[index] = unit_vec(word)
           
    first_neutral_debias = np.asarray(first_neutral_debias)
    # print(first_neutral_debias.shape, first_neutral_debias.shape == embedding_neutral.shape)
    #double_debias = np.zeros((words.shape))
    #for index,word in enumerate(first_debias):
    # double_debias[index] = hard_debias(word)
    
    # apply HardDebias to all neutral, once debiased, words
    double_neutral_debias = hard_debias_2(first_neutral_debias)
    
    double_debias = first_debias.copy()
    
    for index, word in enumerate(double_neutral_debias):
        double_debias[id_neutral[index]] = word

    return double_debias


In [76]:
def unit_vec(vector):
    """calculates unit vector of passed vector"""
    
    unit = np.linalg.norm(vector)
    if unit != 0 and np.isnan(unit) == False :
        return vector/unit
    return vector   

In [77]:
#Gender alignment accuracy/ Neighborhood Metric:
def align_acc(males, females, k=2):
    """bias measurement using KMeans Clustering
    
    takes
    males: male words' embeddings
    females: female words'embeddings
    k: number of clusters to create
    ground truth labels:
    0 = male,
    1 = female
    
    returns
    alignment: alignment accuracy after clustering the embeddings according to gender
    """
    # print(males.shape, females.shape)
    array_m_f = np.concatenate((males,females))
    #print(array_m_f)
    
    #need: k (=1000) most biased female and male word's embedding (cosine similarity embedding & gender direction),
    # perform KMeans on embeddings with k=2
    kmeans = KMeans(n_clusters=k).fit(array_m_f)
    split = males.shape[0]
    
    # print(kmeans.labels_)
    # print(split)
    correct = 0
    #print(kmeans.labels_)
   
    # compute alignment score: cluster assignment vs ground truth gender label
    for i in range(array_m_f.shape[0]):
        # correct clustering if word was assigned its ground truth label
        if i < split and kmeans.labels_[i] == 0:
            correct+= 1
        elif i >= split and kmeans.labels_[i] == 1:
            correct += 1
    
    # alignment score = max(a, 1-a)
    alignment = 1/(2*array_m_f.shape[0]) * correct
    alignment = np.maximum(alignment, 1-alignment)
    
    return alignment 

Additional inputs: words to neutralize $N\subseteq W$, family of equality sets $\mathcal{E} = \{E_1, E_2, ..., E_m\}$ where each $E_i \subseteq W$. For each word $w \in N$, let $\vec{w}$ be re-embedded to $\vec{w}:=(\vec{w}-\vec{w}_B/||\vec{w}-\vec{w}_B||$. For each set $E\in \mathcal{E}$, let $\mu:=\sum_{w\in E}w/|E|$ and $v:=\mu-\mu_B$. For each $w \in E$, $\vec{w}:=v+\sqrt{1-||v||^2}\frac{\vec{w}_B-\mu_B}{||\vec{w}_B-\mu_B||}$. Finally, output the subspace $B$ and the new embedding $\{\vec{w}\in\mathbb{R}^d\}_{w\in W}$.

In [78]:
def hard_debias (word_emb, equality_sets=equalize_pairs, B=gender_subspace):
    """performs hard debias on a word embedding to neutralize it,
    
    takes 
    word_emb: word embedding of the word to be neutralized,
    equalize_pairs: equality pairs, each neutral word should be equidistant to all words in each equality set
    B: the bias subspace
    
    returns
    B: the bias subspace
    new_word_emb: the new embedding for word_emb
    """
    
    # if word_emb is a single embedding:
        # w_orth(word_emb)
        
    # if word_emb is the embeddings of all words to be neutralized:
    
    new_word_emb = np.zeros((word_emb.shape))
    for i, embedding in enumerate(word_emb):
        new_word_emb[i] = w_orth(embedding, B)
        
    for equal_set in equality_sets:
        if equal_set[0] in w2id and equal_set[1] in w2id:
            mean = (embed(equal_set[0]) + embed(equal_set[1])) / 2
            mean_biased = mean - w_orth(mean, B)
            v = mean - mean_biased # what is the biased mean?
            for word in equal_set:
                # print(word)
                word_biased = embed(word) - w_orth(embed(word), B)
                # new_embed = v + np.sqrt(1 - (np.linalg.norm(v)) ** 2) * ((word_biased - mean_biased) / unit_vec(word_biased - mean_biased))
                new_embed = v * ((word_biased - mean_biased) / unit_vec(word_biased - mean_biased))
    
    return new_word_emb#, B im paper steht, dass auch B returned werden soll, aber das macht hier keinen Sinn

In [79]:
def hard_debias_2 (word_emb, equality_sets=female_male_pairs, B=gender_subspace):
    """performs hard debias on a word embedding to neutralize it,
    
    takes 
    word_emb: word embedding of the word to be neutralized,
    equalize_pairs: equality pairs, each neutral word should be equidistant to all words in each equality set
    B: the bias subspace
    
    returns
    B: the bias subspace
    new_word_emb: the new embedding for word_emb
    """
    
    # if word_emb is a single embedding:
        # w_orth(word_emb)
        
    # if word_emb is the embeddings of all words to be neutralized:
    
    new_word_emb = np.zeros((word_emb.shape))
    for i, embedding in enumerate(word_emb):
        new_word_emb[i] = w_orth(embedding, B)
        
    for equal_set in equality_sets:
        if equal_set[0] in w2id and equal_set[1] in w2id:
            mean = (embed(equal_set[0]) + embed(equal_set[1])) / 2
            mean_biased = mean - w_orth(mean, B)
            v = mean - mean_biased # what is the biased mean?
            # print(v)
            for word in equal_set:
                # print(word)
                word_biased = embed(word) - w_orth(embed(word), B)
                # print(np.linalg.norm(v))
                # new_embed = v + np.sqrt(1 - (np.linalg.norm(v)) ** 2) * ((word_biased - mean_biased) / unit_vec(word_biased - mean_biased))
                new_embed = v * ((word_biased - mean_biased) / unit_vec(word_biased - mean_biased))
                
    return new_word_emb#, B im paper steht, dass auch B returned werden soll, aber das macht hier keinen Sinn

In [80]:
def w_orth (word_emb, direction):
    """
    removes direction from word embedding by calculating the orthogonal word vector
    
    w_orth = w - (projection of w onto direction)
    w_orth = w - (direction * (w dot direction))
    
    takes 
    word_emb: word to remove the direction from
    direction: the direction to remove
    
    returns
    unit vector embedding orthogonal to direction   
    """
    
    # formula from Bolukbasi et al. (2016)
    new_word = word_emb - ((word_emb.dot(direction)) * direction)
    

    # print(new_word_emb_two, new_word_emb, new)
    #print(new.shape)
    
    return unit_vec(new_word)

In [81]:
result_equal = double_hard_debias(embedding, embedding_neutral, id_neutral, index_m, index_f)

eval: [0.786, 0.7845, 0.763, 0.7565, 0.74, 0.753, 0.7615, 0.739, 0.739, 0.761, 0.7605, 0.761, 0.739, 0.758, 0.7615, 0.757, 0.7605, 0.7615, 0.7404999999999999, 0.761]
7


In [82]:
print(result_equal)

[[ 0.06391201 -0.06563438 -0.00239887 ...  0.0086454   0.0736558
  -0.0166379 ]
 [ 0.04399874  0.00712372  0.03884452 ...  0.01346767  0.06413636
  -0.07343301]
 [ 0.01482999 -0.00655061  0.06630689 ...  0.01277236  0.02397485
  -0.02752437]
 ...
 [-0.04152475  0.00393666 -0.13917012 ... -0.02381067  0.01355542
   0.07963782]
 [-0.05067399 -0.06722245  0.07248474 ... -0.0060112  -0.04362588
   0.0064206 ]
 [ 0.02887545 -0.04942778  0.02287654 ... -0.02678547  0.01743137
  -0.03597922]]


In [83]:
print(result_equal.shape)

(72306, 300)


In [84]:
result_fem_male = double_hard_debias(embedding, embedding_neutral, id_neutral, index_m, index_f)

eval: [0.7875, 0.715, 0.7625, 0.7565, 0.7384999999999999, 0.747, 0.7375, 0.762, 0.7615, 0.7585, 0.7395, 0.76, 0.762, 0.7585, 0.759, 0.7444999999999999, 0.761, 0.7585, 0.742, 0.739]
1


In [85]:
print(result_fem_male)

[[ 0.05877288 -0.06798788  0.00090709 ...  0.00863048  0.0853816
  -0.01082865]
 [ 0.03183749  0.00894463  0.046105   ...  0.01855882  0.0675874
  -0.06041735]
 [ 0.00377671 -0.00849032  0.07394014 ...  0.01467634  0.03995251
  -0.01584751]
 ...
 [-0.04127628  0.00191055 -0.13965123 ... -0.02568038  0.02017007
   0.07948502]
 [-0.052175   -0.07313977  0.07370454 ... -0.01020328 -0.02606924
   0.00760593]
 [ 0.01788982 -0.04848058  0.02968068 ... -0.0227031   0.02349105
  -0.02431481]]


In [86]:
print(result_equal == result_fem_male)

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]]


### Word Embedding Association Test
Implementation taken from "https://github.com/shivaomrani/HumanBiasInSemantics". 
With minor adjustments such as variable names for readability, see file "weat.py".

In [60]:
# Career and family
# Change from Bill to Tom as in paper to avoid ambiguity
male_names = ["john", "paul", "mike", "kevin", "steve", "greg", "jeff", "tom"]
female_names = ["amy", "joan", "lisa", "sarah", "diana", "kate", "ann", "donna"]
career_attributes = ["executive", "management", "professional", "corporation", "salary", "office", "business", "career"]
family_attributes = ["home", "parents", "children", "family", "cousins", "marriage", "wedding", "relatives"]

In [61]:
# Math and arts
math_words = ["math", "algebra", "geometry", "calculus", "equations", "computation", "numbers", "addition"]
arts_words1 = ["poetry", "art", "dance", "literature", "novel", "symphony", "drama", "sculpture"]
male_attributes1 = ["male", "man", "boy", "brother", "he", "him", "his", "son"]
female_attributes1 = ["female", "woman", "girl", "sister", "she", "her", "hers", "daughter"]

In [62]:
# Science and arts
science_words = ["science", "technology", "pyhsics", "chemistry", "einstein", "nasa", "experiment", "astronomy"]
arts_words2 = ["poetry", "art", "shakespeare", "dance", "literature", "novel", "symphony", "drama"]
male_attributes2 = ["brother", "father", "uncle", "grandfather", "son", "he", "his", "him"]
female_attributes2 = ["sister", "mother", "aunt", "grandmother", "daughter", "she", "hers", "her"]

In [87]:
iterations = 100000
wea_test = weat(male_names, female_names, career_attributes, family_attributes, iterations, result_equal, w2id)
pvalue, effect_size, sd = wea_test.getPValueAndEffect()
print("p-value: ", pvalue)
print("effect size: ", effect_size)
print("standard deviation: ", sd)

The difference of means is  0.06490189518884927
Generating null distribution...
Number of permutations  100000
Getting the entire distribution
p-value:  0.0014929757150399237
effect size:  1.4876466581210122
standard deviation:  0.02184173905370934


### Word Analogy

In [88]:
import analogy_tasks as ana

In [89]:
ana.evaluate_analogy_google(embedding, vocab, w2id)

capital-common-countries.txt:
ACCURACY TOP1: 98.95% (283/286)
capital-world.txt:
ACCURACY TOP1: 92.83% (1774/1911)
currency.txt:
ACCURACY TOP1: 6.49% (27/416)
city-in-state.txt:
ACCURACY TOP1: 77.87% (1921/2467)
family.txt:
ACCURACY TOP1: 68.97% (349/506)
gram1-adjective-to-adverb.txt:
ACCURACY TOP1: 8.87% (88/992)
gram2-opposite.txt:
ACCURACY TOP1: 22.22% (168/756)
gram3-comparative.txt:
ACCURACY TOP1: 78.45% (1045/1332)
gram4-superlative.txt:
ACCURACY TOP1: 45.27% (421/930)
gram5-present-participle.txt:
ACCURACY TOP1: 47.63% (503/1056)
gram6-nationality-adjective.txt:
ACCURACY TOP1: 90.81% (1452/1599)
gram7-past-tense.txt:
ACCURACY TOP1: 34.87% (544/1560)
gram8-plural.txt:
ACCURACY TOP1: 71.62% (954/1332)
gram9-plural-verbs.txt:
ACCURACY TOP1: 45.44% (369/812)
Questions seen/total: 81.64% (15955/19544)
Semantic accuracy: 77.94%  (4354/5586)
Syntactic accuracy: 53.47%  (5544/10369)
Total accuracy: 62.04%  (9898/15955)


In [90]:
ana.evaluate_analogy_google(result_equal, vocab, w2id)

capital-common-countries.txt:
ACCURACY TOP1: 96.15% (275/286)
capital-world.txt:
ACCURACY TOP1: 96.91% (1852/1911)
currency.txt:
ACCURACY TOP1: 6.49% (27/416)
city-in-state.txt:
ACCURACY TOP1: 73.65% (1817/2467)
family.txt:
ACCURACY TOP1: 86.76% (439/506)
gram1-adjective-to-adverb.txt:
ACCURACY TOP1: 20.36% (202/992)
gram2-opposite.txt:
ACCURACY TOP1: 29.50% (223/756)
gram3-comparative.txt:
ACCURACY TOP1: 85.51% (1139/1332)
gram4-superlative.txt:
ACCURACY TOP1: 55.16% (513/930)
gram5-present-participle.txt:
ACCURACY TOP1: 62.22% (657/1056)
gram6-nationality-adjective.txt:
ACCURACY TOP1: 92.12% (1473/1599)
gram7-past-tense.txt:
ACCURACY TOP1: 56.60% (883/1560)
gram8-plural.txt:
ACCURACY TOP1: 78.23% (1042/1332)
gram9-plural-verbs.txt:
ACCURACY TOP1: 58.50% (475/812)
Questions seen/total: 81.64% (15955/19544)
Semantic accuracy: 78.95%  (4410/5586)
Syntactic accuracy: 63.72%  (6607/10369)
Total accuracy: 69.05%  (11017/15955)


In [91]:
ana.evaluate_analogy_msr(embedding, vocab, w2id)

5252
ACCURACY TOP1-MSR: 53.54% (2812/5252)


In [92]:
ana.evaluate_analogy_msr(result_equal, vocab, w2id)

5252
ACCURACY TOP1-MSR: 61.73% (3242/5252)
